In [20]:
import os
import glob
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from define import *
from tqdm import tqdm
from scipy import stats
from scipy.signal import find_peaks

In [21]:
from scipy import signal
import matplotlib.pyplot as plt
from math import pi

### basic data eda

In [22]:
detail = pd.read_csv('C:/Users/user/Desktop/eeg/detail3.csv')
detail['relax'] = detail['after']-detail['before']

###  band filter

In [ ]:
fs = 1000   
num_taps = 101  
alpha = signal.firwin(num_taps, [8, 12] , fs=fs)
beta = signal.firwin(num_taps, [12,30], fs=fs)
gamma = signal.firwin(num_taps, 30 , fs=fs)
delta = signal.firwin(num_taps, [0.5,4], fs=fs)
theta = signal.firwin(num_taps, [4,8], fs=fs)
# y2 = signal.convolve(y, filter, mode='same')

In [23]:
id = ['f','m','p','s']
order = ['before','test','after']
aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
bands = [alpha,beta,gamma,delta,theta]
bands_str = ['alpha','beta','gamma','delta','theta']

part 1

In [ ]:
step_size,window_size = 40,80

for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/raw_data/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            
            X['index'] = X.index    
            X.to_csv('C:/Users/user/Desktop/eeg/feature/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

part 2

In [ ]:
step_size,window_size = 40,80

for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/raw_data/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            
            if o == 'test':
                df = detail[detail['id']== x]
                X['id'] = df['id'].tolist()[n]
                X['light'] = df['light'].tolist()[n] 
                X['music'] = df['music'].tolist()[n]   
                X['before'] = df['before'].tolist()[n] 
                X['after'] = df['after'].tolist()[n]
                X['date'] = df['date'].tolist()[n]  
            elif o == 'before':
                df = detail[detail['id']== x]
                df = df[df.index % 2 == 1]
                X['id'] = df['id'].tolist()[n]
                X['light'] = df['light'].tolist()[n] 
                X['music'] = df['music'].tolist()[n]
                X['date'] = df['date'].tolist()[n]
                X['relax'] = df['before'].tolist()[n]
                X['after_relax'] = df['relax'].tolist()[n]
            else:
                df = detail[detail['id']== x]
                df = df[df.index % 2 == 0]
                X['id'] = df['id'].tolist()[n]
                X['light'] = df['light'].tolist()[n] 
                X['music'] = df['music'].tolist()[n]
                X['date'] = df['date'].tolist()[n]
                X['relax'] = df['after'].tolist()[n]
                X['before_relax'] = df['relax'].tolist()[n]

            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/feature/'+ o +'/'+ x +'/second_' + files[n] ,index=False)

###  bands_raw_data

In [ ]:
# root = 'C:/Users/user/Desktop/eeg/bands_raw_data/'
# os.chdir(root)

# for i in bands:
#     os.mkdir(os.path.join(root,i))
#     for j in order:
#         os.mkdir(os.path.join(root,i,j))
#         for x in id:
#             os.mkdir(os.path.join(root,i,j))

In [ ]:
# for b in range(len(bands)):
#     print(bands_str[b] + ' bands loading.....')
#     filter = bands[b]

#     for o in order:
#         print(o + ' loading.....')

#         for x in tqdm(id):
#             print(x + ' loading.....')
#             root = 'C:/Users/user/Desktop/eeg/raw_data/'+ o +'/'+ x +'/'
#             paths =  os.listdir(root)

#             for p in paths:
#                 path = os.path.join(root,p)
#                 df = pd.read_csv(path)
#                 df.sort_values(by=['timestamps'])
#                 df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
#                 y1 = df['af7'].tolist()
#                 y2 = df['af8'].tolist()
#                 y3 = df['tp9'].tolist()
#                 y4 = df['tp10'].tolist()
#                 y5 = df['right_aux'].tolist()
#                 df[bands_str[b]+'_af7'] = signal.convolve(y1, filter, mode='same')
#                 df[bands_str[b]+'_af8'] = signal.convolve(y2, filter, mode='same')
#                 df[bands_str[b]+'_tp9'] = signal.convolve(y3, filter, mode='same')
#                 df[bands_str[b]+'_tp10'] = signal.convolve(y4, filter, mode='same')
#                 df[bands_str[b]+'_right_aux'] = signal.convolve(y5, filter, mode='same')
#                 df.drop(['af7','af8','tp9','tp10','right_aux'],axis=1)
#                 df.to_csv('C:/Users/user/Desktop/eeg/bands_raw_data/'+ bands_str[b] +'/'+ o +'/'+ x +'/' + p,index=False)


part1

In [ ]:
# root = 'C:/Users/user/Desktop/eeg/bands_feature/'
# os.chdir(root)

# os.mkdir()

# for i in bands:
#     os.mkdir(os.path.join(root,i))
#     for j in order:
#         os.mkdir(os.path.join(root,i,j))
#         for x in id:
#             os.mkdir(os.path.join(root,i,j,x))

In [ ]:
step_size,window_size = 40,80



for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/alpha/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/alpha/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80



for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/beta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/beta/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80



for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/delta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/delta/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80



for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/gamma/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/gamma/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80



for o in order:
    print(o+' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/theta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['mean_'+aisle[l]] = mean_col(lists[l])
                X['std_'+aisle[l]] = std_col(lists[l])
                X['abd_'+aisle[l]] = abd_col(lists[l])
                X['min_'+aisle[l]] = min_col(lists[l])
                X['max_'+aisle[l]] = max_col(lists[l])
                X['range_'+aisle[l]] = range_col(lists[l])
                X['mid_'+aisle[l]] = mid_col(lists[l])
                X['mad_'+aisle[l]] = mad_col(lists[l])
                X['IQR_'+aisle[l]] = IQR(lists[l])
                X['neg_num_'+aisle[l]] = neg_num(lists[l])
                X['pos_num_'+aisle[l]] = pos_num(lists[l])
                X['above_mean_'+aisle[l]] = above_mean(lists[l])
                X['peak_num_'+aisle[l]] = peak_num(lists[l])
                X['skewness_'+aisle[l]] = skewness(lists[l])
                X['kurtosis_'+aisle[l]] = kurtosis(lists[l])
                X['energy_'+aisle[l]] = energy(lists[l])
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/theta/'+ o +'/'+ x +'/first_' + files[n] ,index=False)

part2

In [ ]:
step_size,window_size = 40,80

# for b in range(len(bands_str)):
# print(bands_str[b] +' part is loading=================================================')

for o in order:
    print(o +' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/alpha/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/alpha/'+ o +'/'+ x +'/second_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80

# for b in range(len(bands)):
#     print(bands_str[b] +' part is loading=================================================')

for o in order:
    print(o +' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/beta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/beta/'+ o +'/'+ x +'/second_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80

# for b in range(len(bands)):
#     print(bands_str[b] +' part is loading=================================================')

for o in order:
    print(o +' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/gamma/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/gamma/'+ o +'/'+ x +'/second_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80

# for b in range(len(bands)):
#     print(bands_str[b] +' part is loading=================================================')

for o in order:
    print(o +' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/delta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/delta/'+ o +'/'+ x +'/second_' + files[n] ,index=False)

In [ ]:
step_size,window_size = 40,80

# for b in range(len(bands)):
#     print(bands_str[b] +' part is loading=================================================')

for o in order:
    print(o +' part is loading=================================================')

    for x in id:
        root = 'C:/Users/user/Desktop/eeg/bands_raw_data/theta/'+ o +'/'+ x +'/'
        files = os.listdir(root)
        paths = [os.path.join(root,file) for file in files]
        print('id '+ x + ' is loading...')
        
        for n in tqdm(range(len(paths))):
            df = pd.read_csv(paths[n])  
            df.sort_values(by=['timestamps'])
            df.rename(columns={'Unnamed: 1':'af7','Unnamed: 2':'af8','Unnamed: 3':'tp9','Unnamed: 4':'tp10','Unnamed: 5':'right_aux'}, inplace=True)
            a_list,b_list,c_list,d_list,e_list = [],[],[],[],[] 
                
            for i in range(0, df.shape[0] - window_size, step_size):
                a = df['af7'].values[i: i + 80]
                b = df['af8'].values[i: i + 80]
                c = df['tp9'].values[i: i + 80]
                d = df['tp10'].values[i: i + 80]
                e = df['right_aux'].values[i: i + 80]
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                d_list.append(d)
                e_list.append(e)
                a_list_fft = pd.Series(a_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                b_list_fft = pd.Series(b_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                c_list_fft = pd.Series(c_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                d_list_fft = pd.Series(d_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])
                e_list_fft = pd.Series(e_list).apply(lambda x: np.abs(np.fft.fft(x))[1:41])

            X = pd.DataFrame()
            lists = [a_list,b_list,c_list,d_list,e_list]
            fft_lists = [a_list_fft,b_list_fft,c_list_fft,d_list_fft,e_list_fft]

            aisle = ['aisle1','aisle2','aisle3','aisle4','aisle5']
            for l in range(len(lists)):
                X['fft_mean_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_std_'+aisle[l]] = std_col(fft_lists[l])
                X['fft_abd_'+aisle[l]] = abd_col(fft_lists[l])
                X['fft_min_'+aisle[l]] = min_col(fft_lists[l])
                X['fft_max_'+aisle[l]] = max_col(fft_lists[l])
                X['fft_range_'+aisle[l]] = mean_col(fft_lists[l])
                X['fft_mid_'+aisle[l]] = mid_col(fft_lists[l])
                X['fft_mad_'+aisle[l]] = mad_col(fft_lists[l])
                X['fft_IQR_'+aisle[l]] = IQR(fft_lists[l])
                X['fft_above_mean_'+aisle[l]] = above_mean(fft_lists[l])
                X['fft_peak_num_'+aisle[l]] = peak_num(fft_lists[l])
                X['fft_skewness_'+aisle[l]] = skewness(fft_lists[l])
                X['fft_kurtosis_'+aisle[l]] = kurtosis(fft_lists[l])
                X['fft_energy_'+aisle[l]] = energy(fft_lists[l])
                X['argmax_'+aisle[l]] = argmax(fft_lists[l])
                X['argmin_'+aisle[l]] = argmin(fft_lists[l])
                X['fft_argmax_'+aisle[l]] = argmax_fft(fft_lists[l])
                X['fft_argmin_'+aisle[l]] = argmin_fft(fft_lists[l]) 
            
            X['index'] = X.index
            X.to_csv('C:/Users/user/Desktop/eeg/bands_feature/theta/'+ o +'/'+ x +'/second_' + files[n] ,index=False)